# Validator Delegation Statistics
*18-10-2022*

[onomy-testnet-1 @ Block height 141579]

In [1]:
import pandas as pd
import os
from typing import List
from IPython.display import display

In [2]:
def get_address(moniker :str) -> str:
    """
    Returns valoper address for validator moniker
    """
    for validator in [v for v in vals if v['moniker'] == moniker]:
        return validator['address']

def compute_delegations(moniker: str) -> (int, float):
    """
    Computes delegation statistics
    Returns delegation count and amount in NOM
    """
    header_row = ['delegate', 'amount']
    valoper_csv = get_address(moniker)[5:] + "_dlgs.csv"
    try:
        valoper_delegates = pd.read_csv(valoper_csv, header=None, names=header_row)
    except Exception:
        # Skip missing csv
        return (0, 0, 0)
    # Convert amount to float (int too large)
    valoper_delegates['amount'] = valoper_delegates['amount'].astype(float)
    # Sort descending by amount, show top largest delegates
    df = valoper_delegates.sort_values(['amount'], ascending=[False])
    # Generate readable amount format based on 18 decimal places
    df['nom'] = df['amount'] / 10**18
    df.nlargest(n=10, columns=['amount'])
    num_delegations = df['delegate'].count()
    if num_delegations == 0:
        return (0, 0, 0)
    nom_delegated = df['nom'].sum()
    print(
    f"""{moniker}\n{'='*len(moniker)}
- {get_address(moniker)}
- Total Delegations   : {num_delegations}
- Total NOM delegated : {nom_delegated}\n---\n""")
    if num_delegations > 0:
        top10df = df.nlargest(n=10, columns=['amount'])
        # Compute proportion from top 10 delegators
        top10pc = top10df['nom'].sum() / nom_delegated
        print(f"Top 10 delegators (Share of delegations: {top10pc*100:.4}%):")
        display(top10df)
    else:
        top10pc = 0
    return (num_delegations, nom_delegated, top10pc)


In [3]:
cwd = os.getcwd()
validators_csv :str = "validators.csv"
delegate_csvs :List[str] = [f for f in os.listdir(cwd) if f.startswith("valoper") and f.endswith(".csv")]
xcva : List[str] = ['nomblocks.io', 'lila.holdings', 'NCC', 'NomAddict-JP']

# Read validators
_header_row : List[str]=['moniker', 'address']
vals = pd.read_csv(validators_csv, header=None, names=_header_row).to_dict('records')

# Read blockheight
try:
    with open('blockheight.txt') as f:
        block_height = f.read().strip()
except Exception as e:
    print(e)
    block_height = ""
# Compute delegation statistics
for validator in vals.copy():
    (num, nom, top10pc) = compute_delegations(validator['moniker'])
    if num == 0:
        # Remove validator with empty delegations
        print(f"Skipping validator {validator['moniker']}: empty delegations\n---")
        vals.remove(validator)
    else:
        validator['delegators'] = num
        validator['total'] = nom
        validator['top10pc'] = top10pc * 100
        if validator['moniker'] in xcva:
            validator['xcva'] = True
        else:
            validator['xcva'] = False


Citadel.one
- onomyvaloper1repmzjp6q4ly5hffu4pn6hucwzgyxg86gzr0ye
- Total Delegations   : 3525
- Total NOM delegated : 1493856.6033093275
---

Top 10 delegators (Share of delegations: 40.34%):


,delegate,amount,nom
393,onomy1repmzjp6q4ly5hffu4pn6hucwzgyxg8659pcc0,2.250000e+23,225000.000000
3006,onomy1mgy304lg6zgz383xvkwyt6cjy943up6fffnrft,1.200000e+23,120000.000000
371,onomy1r07fekzzxm8ulanyhvdj3e7pm5sz77k92h7v78,7.737782e+22,77377.818960
263,onomy1z5jrnd3e64uuxykfp9x0jqe70y09w8pxp6fj92,3.500000e+22,35000.000000
2325,onomy14yky73x0qpqsmr3hlcv2fnzaj8drjqzu64f52t,3.490979e+22,34909.788755
2277,onomy154wg9zlte8f4mr6p9au7j5zyau8zm8ey7fe4a3,2.922657e+22,29226.573089
3226,onomy1a82nkg7d6x6l78rgl2urre6xx462arrhql5nmj,2.105156e+22,21051.555462
2164,onomy1nktzmmqfxjqdc3nwv7uvwxlaq37n0k9n6geyjj,2.000000e+22,20000.000000
3176,onomy1uax8gv9um6suxms8pq38h9gpsnx5k9d2ytgyds,2.000000e+22,20000.000000
3117,onomy1uv3y06u3qt80u3u5av57ukfv4s56hmge53qrcy,1.998500e+22,19985.000000


Cosmostation
- onomyvaloper1x5wgh6vwye60wv3dtshs9dmqggwfx2ld4x6cpu
- Total Delegations   : 1650
- Total NOM delegated : 730521.4163673316
---

Top 10 delegators (Share of delegations: 72.55%):


,delegate,amount,nom
334,onomy1x5wgh6vwye60wv3dtshs9dmqggwfx2ldfpc0a2,2.400000e+23,240000.000000
131,onomy1z5jrnd3e64uuxykfp9x0jqe70y09w8pxp6fj92,1.375668e+23,137566.832318
1403,onomy1mgy304lg6zgz383xvkwyt6cjy943up6fffnrft,1.000000e+23,100000.000000
1242,onomy1cgfm256d0gk2mthn7l4cdnhmyh74arjlsa8euc,1.220787e+22,12207.868868
720,onomy1wyerwl5hdvcdsq9qswe2q4zfd99lzupx4ew5c6,1.000000e+22,10000.000000
1629,onomy1lj8y739w23xjmq7ypt8pycyc5egpqgq862m3n8,9.870000e+21,9870.000000
964,onomy1j6y377n4x2e8nzycp3vrvejfawvrkjepz8ag8a,5.700000e+21,5700.000000
1257,onomy1c35pghdm9cl2prtd8jxgf02dktuczwm6ylcgk2,5.000000e+21,5000.000000
1173,onomy1k5vnnt375wfpdy0guumjku6a95t5w9y887wy73,5.000000e+21,5000.000000
1428,onomy1mceehqeuc56pmjp4alcrzqnzyj5a36wv2w590w,4.628757e+21,4628.756769


lila.holdings
- onomyvaloper12reepqzweq5hjd3lxaqlfh0f6pumz0dt0vq97y
- Total Delegations   : 1081
- Total NOM delegated : 571313.3413965093
---

Top 10 delegators (Share of delegations: 80.46%):


...

,delegate,amount,nom
1563,onomy1cd44n385368wm6sdzualuf634mfqnq2ufn99gw,2.400000e+23,240000.000000
1746,onomy1mgy304lg6zgz383xvkwyt6cjy943up6fffnrft,1.500000e+23,150000.000000
182,onomy1z5jrnd3e64uuxykfp9x0jqe70y09w8pxp6fj92,2.500000e+22,25000.000000
215,onomy1rqg87xgeymk28r5seadgjl9fk0a7k3u9axqty8,9.046118e+21,9046.117774
1288,onomy1nktzmmqfxjqdc3nwv7uvwxlaq37n0k9n6geyjj,9.000000e+21,9000.000000
1497,onomy1hxcfp67t9cwdt29fex5t07u9h7ur3ak2hty2jw,5.830000e+21,5830.000000
439,onomy1xdgn48r6ef0s7l6q3l87s9zttsxf64ycj4njr8,3.381025e+21,3381.024665
1032,onomy10jath6g7kn8ly6jkthdrpu37dd72s565z0u88d,2.900000e+21,2900.000000
29,onomy1q2kym8shnecay9n2fgl99cmc6jl4na9qmdqxq2,2.500000e+21,2500.000000
451,onomy1xna37ta0wmkjxxf9qw9l2e4976laq6c4r3rzpu,2.250000e+21,2250.000000


NCC
===
- onomyvaloper164fv330dfr4d9z8q0eg0ne4fal687f4m3nlql4
- Total Delegations   : 2741
- Total NOM delegated : 489864.0560142083
---

Top 10 delegators (Share of delegations: 76.49%):


,delegate,amount,nom
2321,onomy164fv330dfr4d9z8q0eg0ne4fal687f4md5ahrr,2.415459e+23,241545.940722
2367,onomy1mgy304lg6zgz383xvkwyt6cjy943up6fffnrft,1.000187e+23,100018.721981
200,onomy1z5jrnd3e64uuxykfp9x0jqe70y09w8pxp6fj92,1.000000e+22,10000.000000
1313,onomy108k9l8w70j0yaxua26r9jmsmvvues43h6ylwzx,5.306510e+21,5306.510315
539,onomy1xdgn48r6ef0s7l6q3l87s9zttsxf64ycj4njr8,4.463345e+21,4463.345021
2181,onomy1erjsgqhqlsqhhx8vgvkrvy3jcf5ypy69ht5k92,3.949515e+21,3949.515292
1716,onomy1n36hvgfh9srmj3q6v3egcap0fpmj6czzupfqc4,3.205552e+21,3205.551798
611,onomy18ghf4thvfq4ce4j4we6ngur2elygcme8sgyjx4,2.900000e+21,2900.000000
1711,onomy1n0zmxzxjmddudvlahfac7clxfhynllxy5x65at,1.802111e+21,1802.110760
2491,onomy1u5x4kcs9wp6shurx37j7ppppgga9aktdp0m0n7,1.500000e+21,1500.000000


Skipping validator NomAddict: empty delegations
---
The-Tao
- onomyvaloper1ugfs6cu2yk4vvh8hs6g6alufjyxwehn6uxpg97
- Total Delegations   : 363
- Total NOM delegated : 397196.40981231944
---

Top 10 delegators (Share of delegations: 96.81%):


,delegate,amount,nom
317,onomy1ugfs6cu2yk4vvh8hs6g6alufjyxwehn6qprleg,2.400000e+23,240000.000000
303,onomy1mgy304lg6zgz383xvkwyt6cjy943up6fffnrft,1.100000e+23,110000.000000
178,onomy1s9vhuy9nefpga4ks0m4k88dzt3ej30jwl3h52k,2.114683e+22,21146.830732
343,onomy1700693g30pnz6klp97lenpyfmk9yajca3wcefa,3.954217e+21,3954.216639
30,onomy1z5jrnd3e64uuxykfp9x0jqe70y09w8pxp6fj92,3.000000e+21,3000.000000
5,onomy1q2kym8shnecay9n2fgl99cmc6jl4na9qmdqxq2,2.500000e+21,2500.000000
11,onomy1qetuyuj5z6afam22sccgnjyzspdy5lpgk45rtt,1.111000e+21,1111.000000
312,onomy1mlzkngm2ermx2ztpdz786ulfuypegj6xrhkg0p,1.000000e+21,1000.000000
255,onomy1hagu7n29da7um6rjlk294f6wc97sd0f32wejpk,9.500000e+20,950.000000
273,onomy1cukcmg0mnzafe3vdsj4sc82z7jcekclahs9adl,8.470161e+20,847.016112


## Validator Summary Statistics [onomy-testnet-1]

In [4]:
# Convert to dataframe, sort and compute percentages
df = pd.DataFrame.from_dict(vals)
df['pc'] = df['total'] / df['total'].sum() * 100
df = df.sort_values(['pc'], ascending=[False])

# Calculate share by XCVA
xcva_total : float = df.loc[df['xcva'] == True, 'total'].sum()
xcva_pc : float = xcva_total / df['total'].sum() * 100

# Display summary
display(df[['moniker', 'address', 'delegators', 'pc', 'total', 'top10pc']].round(2))
print(f"Block Height : {block_height}")
print(f"XCVA share   : {xcva_pc:.4}%")

,moniker,address,delegators,pc,total,top10pc
0,Citadel.one,onomyvaloper1repmzjp6q4ly5hffu4pn6hucwzgyxg86g...,3525,22.37,1493856.60,40.34
3,onomy,onomyvaloper1v92yldfkep9lqj8g4qwa37vywqdpe2zrj...,3262,15.32,1023065.98,55.24
1,Cosmostation,onomyvaloper1x5wgh6vwye60wv3dtshs9dmqggwfx2ld4...,1650,10.94,730521.42,72.55
7,Forbole,onomyvaloper1cd44n385368wm6sdzualuf634mfqnq2u4...,2070,8.89,593595.64,75.79
2,lila.holdings,onomyvaloper12reepqzweq5hjd3lxaqlfh0f6pumz0dt0...,1081,8.56,571313.34,80.46
4,nomblocks.io,onomyvaloper1dq2xznpmvjchcm0nt46xfvyujsvpzveck...,1261,8.01,534720.55,82.62
8,NCC,onomyvaloper164fv330dfr4d9z8q0eg0ne4fal687f4m3...,2741,7.34,489864.06,76.49
6,NomAddict-JP,onomyvaloper15n6larm8wl0l895dh552vhxdm6czjqnhc...,638,7.27,485666.27,94.20
9,The-Tao,onomyvaloper1ugfs6cu2yk4vvh8hs6g6alufjyxwehn6u...,363,5.95,397196.41,96.81
5,Cappuccinode,onomyvaloper1du0tgu0z74ush2mcymg8ttxsx2szne3ju...,338,5.36,358231.78,96.85


Block Height : 141579
XCVA share   : 31.17%
